## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-28-19-51-18 +0000,nypost,Phoenix dad arrested after leaving 4 kids in 1...,https://nypost.com/2025/07/28/us-news/phoenix-...
1,2025-07-28-19-51-11 +0000,bbc,Starmer shares Gaza peace blueprint with Trump,https://www.bbc.com/news/articles/cly6zgyy0wjo...
2,2025-07-28-19-49-44 +0000,nyt,"Who Pays for U.S. Tariffs, and Where Does the ...",https://www.nytimes.com/2025/07/28/business/ec...
3,2025-07-28-19-42-18 +0000,startribune,ICE wants to deport Wisconsin driver charged w...,https://www.startribune.com/ice-wants-to-depor...
4,2025-07-28-19-42-06 +0000,nyt,Manhunt Underway in Arkansas After Couple Are ...,https://www.nytimes.com/2025/07/28/us/arkansas...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2315/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
16,trump,48
218,deal,25
134,trade,23
13,gaza,18
31,killed,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
121,2025-07-28-14-42-00 +0000,wsj,Trump said the U.S. reached a trade deal with ...,https://www.wsj.com/economy/trade/european-uni...,131
42,2025-07-28-18-24-16 +0000,nypost,France presses EU to unload ‘trade bazooka’ on...,https://nypost.com/2025/07/28/business/france-...,124
153,2025-07-28-11-56-35 +0000,nyt,"Europe, Worried by Other Global Crises, Agrees...",https://www.nytimes.com/2025/07/28/world/europ...,111
41,2025-07-28-18-25-00 +0000,wsj,Stock Market Today: Euro Drops After Trump Str...,https://www.wsj.com/livecoverage/stock-market-...,110
233,2025-07-27-22-21-28 +0000,wapo,"Trump, E.U. reach contours of trade deal with ...",https://www.washingtonpost.com/politics/2025/0...,110


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
121,131,2025-07-28-14-42-00 +0000,wsj,Trump said the U.S. reached a trade deal with ...,https://www.wsj.com/economy/trade/european-uni...
71,61,2025-07-28-17-17-00 +0000,wsj,President Trump said he would give Russia’s Vl...,https://www.wsj.com/politics/trump-losing-pati...
143,51,2025-07-28-13-14-04 +0000,nypost,Starving Palestinians swarm aid trucks in Gaza...,https://nypost.com/2025/07/28/world-news/pales...
79,43,2025-07-28-16-50-00 +0000,wsj,Business leaders on both sides of the Atlantic...,https://www.wsj.com/economy/trade/companies-we...
117,38,2025-07-28-15-04-00 +0000,wsj,Former Gov. Roy Cooper of North Carolina launc...,https://www.wsj.com/politics/elections/roy-coo...
41,37,2025-07-28-18-25-00 +0000,wsj,Stock Market Today: Euro Drops After Trump Str...,https://www.wsj.com/livecoverage/stock-market-...
58,37,2025-07-28-18-02-44 +0000,nyt,Thailand and Cambodia Agree to Halt Fighting T...,https://www.nytimes.com/2025/07/28/world/asia/...
11,33,2025-07-28-19-33-00 +0000,wsj,The Trump administration is considering a plan...,https://www.wsj.com/politics/policy/patent-sys...
109,30,2025-07-28-15-26-34 +0000,nyt,Suspect in Traverse City Walmart Stabbing Atta...,https://www.nytimes.com/2025/07/27/us/walmart-...
60,28,2025-07-28-17-56-49 +0000,nypost,Ghislaine Maxwell makes long-shot Supreme Cour...,https://nypost.com/2025/07/28/us-news/ghislain...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
